In [1]:
import numpy as np
import cv2 ,os
import imutils
import csv
import glob
import pandas as pd

In [2]:
#colorDiscripteur
def histogram(image, mask):
    hist = cv2.calcHist([image], [0, 1, 2], mask, (8, 12, 3),[0, 180, 0, 256, 0, 256])
    if imutils.is_cv2():
        hist = cv2.normalize(hist).flatten()
    else:
        hist = cv2.normalize(hist, hist).flatten()

    return hist

def describe(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV) #convertir notre image en espace hsv
    features = []
    (h, w) = image.shape[:2]
    (cX, cY) = (int(w * 0.5), int(h * 0.5))   #calcul du centre de notre image
    segments = [(0, cX, 0, cY), (cX, w, 0, cY), (cX, w, cY, h), (0, cX, cY, h)] #ici on divise l'image en 4 parties/regions pour chaque coté 
    
    for (startX, endX, startY, endY) in segments:
        cornerMask = np.zeros(image.shape[:2], dtype = "uint8")  #construire un mask noir pour chaque region
        cv2.rectangle(cornerMask, (startX, startY), (endX, endY), 255, -1)
        hist = histogram(image, cornerMask)   #extraire notre histogramme 
        features.extend(hist)                # maj de notre vecteur de featues
    hist = histogram(image, cornerMask)     #extraire un histogramme couleur 
    features.extend(hist)     

    return features

In [3]:
def index_train_feature(indexPathTrain):
    databasePathTrain = "asl_alphabet_train"
    output = open(indexPathTrain, "w")
    metadata_i = 1
    metadata_o = ""

    for imagePath in glob.glob(databasePathTrain + os.path.sep + "*" + os.path.sep + "*.*"):
        imageID = imagePath.split(os.path.sep)[-1] # extraire le nom de chaque photo
        target = imagePath.split(os.path.sep)[-2] # extraire le classe de chaque photo

        image = cv2.imread(imagePath)
        features = describe(image)
        features = [str(f) for f in features]

        if metadata_i == 1:
            for i in range(len(features)):
                metadata_o += "X" + str(i+1) + ","
            metadata_i = 0
            metadata_o = metadata_o[:-1]
            output.write("%s,%s,%s\n" % ("imageID", "target", metadata_o))
        output.write("%s,%s\n" % (imageID + "," + target, ",".join(features)))

    output.close()

In [4]:
indexPathTrain = "index_train.csv"

index_train_feature("index_train.csv")

data = pd.read_csv(indexPathTrain)

X_train = data.iloc[:,2:].values
Y_train = data.iloc[:,1].values

del data

In [5]:
def index_test_feature(indexPathTest):
    databasePathTest = "asl_alphabet_test"
    output = open(indexPathTest, "w")
    metadata_i = 1
    metadata_o = ""
    
    for imagePath in glob.glob(databasePathTest + "/*.jpg"):
        imageID = imagePath.split(os.path.sep)[-1]

        image = cv2.imread(imagePath)
        features = describe(image)
        features = [str(f) for f in features]
        
        if metadata_i == 1:
            for i in range(len(features)):
                metadata_o += "X" + str(i+1) + ","
            metadata_i = 0
            metadata_o = metadata_o[:-1]
            output.write("%s,%s\n" % ("imageID", metadata_o))

        output.write("%s,%s\n" % (imageID, ",".join(features)))

    output.close()

In [6]:
indexPathTest = "index_test.csv"

index_test_feature("index_test.csv")

data = pd.read_csv(indexPathTest)

X_test = data.iloc[:,1:].values
Y_test = data.iloc[:,0].values

del data

In [7]:
from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()
X_train = std_scaler.fit_transform(X_train)
X_test = std_scaler.fit_transform(X_test)

In [11]:
from sklearn import svm

clf=svm.SVC()
%time clf.fit(X_train,Y_train)

/home/ayoub_ridouani/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


CPU times: user 1h 15min 50s, sys: 3.31 s, total: 1h 15min 54s
Wall time: 1h 15min 57s


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [12]:
print("score: ", clf.score(X_train,Y_train))

score:  0.9881609195402299


In [13]:
pred = clf.predict(X_test)

for i in range(0,len(Y_test)):
    print(Y_test[i], " => ", pred[i])

J_test.jpg  =>  J
Y_test.jpg  =>  Y
B_test.jpg  =>  B
G_test.jpg  =>  G
X_test.jpg  =>  X
Q_test.jpg  =>  P
L_test.jpg  =>  L
R_test.jpg  =>  R
V_test.jpg  =>  V
H_test.jpg  =>  H
I_test.jpg  =>  I
A_test.jpg  =>  A
T_test.jpg  =>  T
P_test.jpg  =>  P
M_test.jpg  =>  M
space_test.jpg  =>  space
C_test.jpg  =>  C
O_test.jpg  =>  O
N_test.jpg  =>  M
Z_test.jpg  =>  Z
nothing_test.jpg  =>  nothing
D_test.jpg  =>  D
W_test.jpg  =>  W
K_test.jpg  =>  K
U_test.jpg  =>  W
S_test.jpg  =>  S
F_test.jpg  =>  F
E_test.jpg  =>  E
